###### calculate sentiment score, age of content, popularity of content, review keys

In [65]:
# import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import pandas as pd
import pymongo
import re
import numpy as np


In [51]:
pd.options.display.float_format = "{:.2f}".format

In [52]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["contelligenz"]
mycol = mydb["movies"]

In [53]:
scorecastcol = mydb['popscastcores']
scorecrewcol = mydb['popscrewcores']

In [7]:
# mycol.update_many({}, {"$set": {"sentiment-score": 0}}, upsert=False, array_filters=None)
# mycol.update_many({}, {"$set": {"review-keyword": []}}, upsert=False, array_filters=None)
# mycol.update_many({}, {"$set": {"plot-keyword": []}}, upsert=False, array_filters=None)
# mycol.update_many({}, {"$set": {"popularity-score": 0}}, upsert=False, array_filters=None)
# mycol.update_many({}, {"$set": {"cast-score": 0}}, upsert=False, array_filters=None)
# mycol.update_many({}, {"$set": {"crew-score": 0}}, upsert=False, array_filters=None)

In [8]:
# mycol.update_many({}, {"$unset": {"cast-score":1}})
# mycol.update_many({}, {"$unset": {"crew-score":1}})

In [54]:
df_1 = pd.DataFrame(list(mycol.find()))

In [55]:
df_1.columns

Index(['_id', 'title', 'id', 'release', 'genre', 'cast', 'crew-writers',
       'crew-director', 'plot-summary', 'imdb-rating', 'imdb-votes', 'reviews',
       'summaries', 'synopsis', 'sentiment-score', 'popularity-score',
       'review-keyword', 'plot-keyword'],
      dtype='object')

In [56]:
class Scores():
    def __init__(self):

        self.sent = SentimentIntensityAnalyzer()

    def pop_score(self,n_votes,avg_rating,n,m):
        score = ((avg_rating*n_votes) + (c*m)) / (n_votes+c)
        return score
    
    def cast_pop(self, tot_votes,cast_votes):
        score = (cast_votes / (tot_votes-cast_votes)) * 10
        return score
    
    def sentiment_score(self,rev):
        rev = re.sub('(\\n)|(--)|(\\\)|(\?)|(,)','',rev)
        sent = SentimentIntensityAnalyzer()
        score = sent.polarity_scores(rev)
        print(i,score)
        return score
    
    

In [57]:
score_ = Scores()

#### cast score
(how popular the actor is compared to other movies)

$\frac{cast\_votes}{total\_votes-cast\_votes}*10$

here, 
* cast_votes is the total votes of movies recieved by actor for all movies (shows how popular the actor is)
* total_votes - cast_votes is for popularity of other actors.
* 10 is the scaler.

In [58]:
df_1['cast_'] = df_1['cast'].apply(lambda x : ' '.join([i for i in x]))
uniq_cast = df_1['cast'].explode('cast').unique()
print(len(uniq_cast))
print(len(list(set(uniq_cast))))
total_votes = df_1['imdb-votes'].sum()

1181
1181


In [59]:
for i in range(len(uniq_cast)):
    try:
        cast_votes = df_1[df_1['cast_'].str.contains(uniq_cast[i])]['imdb-votes'].sum()
        score = score_.cast_pop(total_votes,cast_votes)
        score = round(score,2)
        print(uniq_cast[i],score,i)
        scorecastcol.insert_one({"name":uniq_cast[i],"score":score})
    except Exception as e:
        print(i,uniq_cast[i])
        

    

0 Aamir Khan
1 Madhavan
2 Sharman Joshi
3 Kareena Kapoor
4 nan
5 Rajinder Sharma Nanu
6 Anushka Sharma
7 Sushant Singh Rajput
8 Fatima Sana Shaikh
9 Sanya Malhotra
10 Sakshi Tanwar
11 Gracy Singh
12 Rachel Shelley
13 Paul Blackthorne
14 Shah Rukh Khan
15 Kajol
16 Katie Amanda Keane
17 Kenton Duty
18 Gayatri Joshi
19 Kishori Ballal
20 Smith Seth
21 Manoj Bajpayee
22 Richa Chadha
23 Nawazuddin Siddiqui
24 Tigmanshu Dhulia
25 Ranbir Kapoor
26 Priyanka Chopra Jonas
27 Ileana D'Cruz
28 Saurabh Shukla
29 Sanjay Dutt
30 Arshad Warsi
31 Boman Irani
32 Salman Khan
33 Harshaali Malhotra
34 Saif Ali Khan
35 Akshaye Khanna
36 Preity Zinta
37 Hrithik Roshan
38 Farhan Akhtar
39 Abhay Deol
40 Katrina Kaif
41 Amrish Puri
42 Farida Jalal
43 Jaya Bachchan
44 Ajay Devgn
45 Shriya Saran
46 Tabu
47 Rajat Kapoor
48 Ranveer Singh
49 Arjun Kapoor
50 Irrfan Khan
51 Kangana Ranaut
52 Rajkummar Rao
53 Lisa Haydon
54 Jeffrey Ho
55 Mohammad Amir Naji
56 Amir Farrokh Hashemian
57 Bahare Seddiqi
58 Nafise Jafar-Moha

#### popularity score 
(how popular a movie is compared to other ratings)

$\frac{(avg\_rating*n\_votes) + (c*m)}{n\_votes+c}$

here, 
* avg_rating is the rating recieved.
* n_votes is the total votes recieved.
* c is the confidence number (calculated as 25% quantile from votes of each movie).
* m is the avg_rating of all movies.

In [61]:
df_1['imdb-votes'] = df_1['imdb-votes'].apply(lambda x: int(''.join(x.split(','))) if x!='N/A' else 0)
df_1['imdb-rating'] = df_1['imdb-rating'].apply(lambda x: float(x)if x!='N/A' else 0)
c = df_1['imdb-votes'].quantile(0.25) # confidence number
m = df_1['imdb-rating'].mean() # avg of rating

In [ ]:
for i in range(len(df_1)):
    try:
        id = df_1.iloc[i]['id']
        n_votes = df_1.iloc[i]['imdb-votes']
        avg_rating = df_1.iloc[i]['imdb-rating']
        score = score_.pop_score(n_votes,avg_rating,c,m)
        score = round(score,2)
        mycol.update_one({"id":id},{"$set":{'popularity-score':score}})
    except Exception as e:
        print(i,e)
        

#### crew scores

In [63]:
df_1['crew'] = df_1['crew-writers'] + df_1['crew-director']
df_1['crew']  = df_1['crew'].apply(lambda x: re.sub(",","",x))
df_1['crew-writers_'] = df_1['crew-writers'].apply(lambda x: x.split(','))
df_1['crew-director_'] = df_1['crew-director'].apply(lambda x : [x])
df_1['crew_']=df_1['crew-director_'] + df_1['crew-writers_']

uniq_crew = df_1['crew_'].explode('crew_').unique()
print(len(uniq_crew))
print(len(list(set(uniq_crew))))
total_votes = df_1['imdb-votes'].sum()

1417
1417


In [ ]:
for i in range(len(uniq_crew)):
    try:
        cast_votes = df_1[df_1['crew'].str.contains(uniq_crew[i])]['imdb-votes'].sum()
        score = score_.cast_pop(total_votes,cast_votes)
        score = round(score,2)
        print(uniq_crew[i],score,i)
        scorecrewcol.insert_one({"name":uniq_crew[i],"score":score})
    except Exception as e:
        print(i,uniq_crew[i])
        

Rajkumar Hirani 0.7 0
 Abhijat Joshi 0.62 1
 Vidhu Vinod Chopra 0.4 2
N/A 1.6 3
Nitesh Tiwari 0.4 4
Piyush Gupta 0.37 5
 Shreyas Jain 0.4 6
 Nikhil Mehrotra 0.38 7
Ashutosh Gowariker 0.22 8
 Kumar Dave 0.11 9
 Sanjay Dayma 0.11 10
Karan Johar 0.28 11
Shibani Bathija 0.14 12
 Niranjan Iyengar 0.19 13
M.G. Sathya 0.09 14
 Ashutosh Gowariker 0.1 15
 Sameer Sharma 0.09 16
Anurag Kashyap 0.33 17
Akhilesh Jaiswal 0.2 18
 Anurag Kashyap 0.3 19
 Sachin K. Ladia 0.19 20
Anurag Basu 0.18 21
 Tani Basu 0.16 22
 Sanjeev Dutta 0.18 23
Vidhu Vinod Chopra 0.49 24
 Rajkumar Hirani 0.11 25
 Lajan Joseph 0.08 26
Kabir Khan 0.12 27
Vijayendra Prasad 0.09 28
 Kabir Khan 0.1 29
 Parveez Sheikh 0.17 30
Farhan Akhtar 0.23 31
Kassim Jagmagia 0.07 32
 Farhan Akhtar 0.11 33
 Karan Vishwanath Kashyap 0.08 34
Zoya Akhtar 0.11 35
 Reema Kagti 0.1 36
 Zoya Akhtar 0.07 37
Aditya Chopra 0.24 38
 Javed Siddiqui 0.12 39
Nikkhil Advani 0.09 40
Niranjan Iyengar 0.26 41
 Karan Johar 0.1 42
Nishikant Kamat 0.1 43
Jeethu Jo

/var/folders/lq/js5qhrxn52j_syhy7sq926s80000gn/T/ipykernel_3467/308809684.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cast_votes = df_1[df_1['crew'].str.contains(uniq_crew[i])]['imdb-votes'].sum()


Prakash Mehra 0.01 862
Chaitanya Tamhane 0.0 863
 Sameer Rajda 0.0 864
Honeypreet Insan, Gurmeet Ram Rahim Singh 0.0 865
Prakash Kovelamudi 0.0 866
Kanika Dhillon 0.02 867
Akshay Roy 0.0 868
Suprotim Sengupta 0.0 869
Basu Chatterjee 0.0 870
 Rohit Malhotra 0.01 871
 Raj Nidimoru 0.02 872
Tapan Sinha 0.0 873
 Hrishikesh Mukherjee 0.0 874
Ajay Bahl 0.01 875
Nila Madhab Panda 0.0 876
 Protiqe Mojoomdar 0.0 877
 Nila Madhab Panda 0.0 878
 V.P. Sathe 0.0 879
Anshai Lal 0.0 880
Nitin Kakkar 0.0 881
Shlok Sharma 0.0 882
 Veeru Potla 0.0 883
Rupinder Chahal 0.0 884
 Rajesh Chawla 0.0 885
 Balwinder Singh Janjua 0.0 886
Farhad Bhiwandiwala 0.0 887
Shonali Bose, Ajeet Ghorpade, Nilesh Maniyar 0.0 888
Shonali Bose 0.01 889
 Nilesh Maniyar 0.01 890
 Atika Chohan 0.0 891
Sunil Dutt 0.0 892
Akhtar-Ul-Iman 0.0 893
Deepak Venkateshan 0.0 894
 Sanjay Gupta 0.0 895
Lalit Marathe 0.0 896
 Anwar Maqsood 0.0 897
Radhika Rao, Vinay Sapru 0.0 898
Radhika Rao 0.0 899
 Vinay Sapru 0.0 900
 Aditya Dhar 0.0 901


#### sentiment scores

In [128]:
final_senti = {
}
for i in range(len(df_1)):
    id = df_1.iloc[i]['id']
    scores = [0]
    tags = {
        'positive':0,
        'negative':0,
        'neutral':0,
        'score':0
    }
    rev = df_1.iloc[i]['reviews']
    sent = SentimentIntensityAnalyzer()
    if type(rev)==list:
        if len(rev)>0:
            for j in rev:
                rev_ = re.sub('(\\n)|(--)|(\\\)|(\?)|(,)','',j)
                score = sent.polarity_scores(rev_)['compound']
                scores.append(score)
                if score!='nan':
                    if score>0.05:
                        tags['positive']+=1
                    elif score<-0.05:
                        tags['negative']+=1
                    else:
                        tags['neutral']+=1
            tags['score'] = np.nanmean(scores)
            title =  df_1.iloc[i]['title']
            final_senti[title] = tags
    mycol.update_one({'id':id},{"$set":{"sentiment-score": tags}})

#### review keyword extraction

In [104]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
import yake

In [105]:

class KeyExtractor():
    def __init__(self):
        self.kw_extractor = yake.KeywordExtractor()
        
    
    def yake_extractor(self,text):
        keywords = self.kw_extractor.extract_keywords(text)
        return keywords
    
    def get_hotwords(self,text):
        result = []
        pos_tag = ['PROPN', 'ADJ'] 
        doc = nlp(text.lower()) 
        for token in doc:
            if(token.text in nlp.Defaults.stop_words or token.text in punctuation or len(token.text)<4):
                continue
            if(token.pos_ in pos_tag):
                result.append(token.text)
            
        most_common_list = Counter(set(result)).most_common(5)
        return most_common_list


In [106]:
k = KeyExtractor()

In [ ]:

for i in range(len(df_1)):
    print(i)
    keys_= []
    rev = df_1.iloc[i]['reviews']
    id = df_1.iloc[i]['id']
    if type(rev)!=float:
        rev = rev[:20]
        for j in rev:
                most_common_list = k.get_hotwords(j)
                keys_.append([g[0] for g in most_common_list])
    keys = list(set([item for sublist in keys_ for item in sublist]))
    mycol.update_one({'id':id}, {"$push":{'review-keyword':{'$each':keys}}})

#### plot keyword extraction

In [ ]:

for i in range(len(df_1)):
    print(i)
    keys_=[]
    rev = df_1.iloc[i]['synopsis']
    id = df_1.iloc[i]['id']
    if type(rev)!=float:
        most_common_list = k.yake_extractor(rev)
        keys_.append([i[0] for i in most_common_list])
    mycol.update_one({'id':id}, {"$push":{'review-keyword':{'$each':most_common_list}}})